# TCC - Análise de Sinais EEG

Este notebook apresenta um fluxo de trabalho completo para a análise de sinais de EEG, desde a leitura e pré-processamento dos dados até a construção, treinamento e avaliação de um modelo de aprendizado profundo para classificação.

## 1. Bibliotecas e Configurações Iniciais

Importação das bibliotecas necessárias e configuração do ambiente, incluindo a alocação de memória da GPU, se disponível.

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import pywt
from scipy.signal import butter, filtfilt, iirnotch
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Configuração da GPU (opcional)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)]
        )
    except RuntimeError as e:
        print(e)

# Caminhos dos arquivos
RAW_MUSE_PATH = '../datalake/raw/Muse-v1.0/MU.txt'
CSV_MUSE_PATH = '../datalake/raw/Muse-v1.0/MU.csv'
PREPROCESSED_MUSE_PATH = '../datalake/processed/Muse-v1.0/MU_filtered.csv'

2025-10-16 07:22:56.372034: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-16 07:22:56.901490: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-16 07:23:00.628377: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## 2. Leitura e Preparação dos Dados

Esta seção é responsável por carregar os dados brutos, convertê-los para um formato CSV mais estruturado e, em seguida, realizar a leitura para o pré-processamento.

In [2]:
def generate_muse_v1_csv(input_path, output_path):
    """
    Converte o arquivo de dados brutos (formato .txt) para um arquivo CSV.
    """
    if not os.path.exists(output_path):
        col_names = ["id", "event", "device", "channel", "code", "size", "data"]
        df = pd.read_csv(input_path, header=None, sep='\t', names=col_names)
        df.to_csv(output_path, index=False, sep=';')
        print(f"Arquivo CSV gerado em: {output_path}")
    else:
        print(f"Arquivo CSV já existe em: {output_path}")

# Gerar o CSV a partir do arquivo .txt
generate_muse_v1_csv(RAW_MUSE_PATH, CSV_MUSE_PATH)

Arquivo CSV já existe em: ../datalake/raw/Muse-v1.0/MU.csv


## 3. Pré-processamento e Filtragem dos Sinais

Aplicação de filtros para remover ruídos e artefatos dos sinais de EEG. As seguintes técnicas são utilizadas:
- **Filtro Butterworth Passa-Alta:** Para remover a flutuação da linha de base.
- **Filtro Notch:** Para remover a interferência da rede elétrica (60 Hz).
- **Denoising com Transformada Wavelet Discreta (DWT):** Para atenuar ruídos de alta frequência.

In [3]:
class Preprocessing:
    def butterworth_highpass(self, data, cutoff, fs, order):
        b, a = butter(order, cutoff / (fs / 2), btype="high", analog=False)
        return filtfilt(b, a, data)

    def notch_filter(self, data, fs, freq, Q):
        b, a = iirnotch(w0=freq/(fs/2), Q=Q)
        return filtfilt(b, a, data)

    def dwt_denoise_reconstruct(self, signal, wavelet='db4', level=3, mode='soft'):
        coeffs = pywt.wavedec(signal, wavelet=wavelet, level=level)
        n = len(signal)
        for i in range(1, len(coeffs)):
            cd = coeffs[i]
            sigma = np.median(np.abs(cd)) / 0.6745 if cd.size > 0 else 0.0
            thresh = sigma * np.sqrt(2 * np.log(n)) if sigma > 0 else 0.0
            coeffs[i] = pywt.threshold(cd, thresh, mode=mode)
        rec = pywt.waverec(coeffs, wavelet=wavelet)
        return np.asarray(rec[:n])

    def read_input(self, input_path:str) -> pd.DataFrame:
        if not os.path.exists(input_path):
            raise FileNotFoundError(f"Arquivo não encontrado: {input_path}")
        return pd.read_csv(input_path, sep=';')

    def execute(self, input_path, output_path):
        df = self.read_input(input_path)
        df = df[df['code'] != -1]
        df_other = df[["event", "device", "code", "size"]].drop_duplicates(subset=["event"])
        df_pivot = df.pivot(index="event", columns="channel", values="data").reset_index()
        CHANNELS = [col for col in df_pivot.columns if col not in ["event"]]
        df_pivot = df_pivot.merge(df_other, on="event", how="inner")
        fs = 220
        filtered = []
        for _, row in df_pivot.iterrows():
            filtered_row = {
                'event': row['event'],
                'device': row['device'],
                'code': row['code'],
                'size': row['size']
            }
            for channel in CHANNELS:
                data = np.array([int(x) for x in row[channel].split(',')], dtype=float)
                data = self.butterworth_highpass(data=data, cutoff=0.1, fs=fs, order=5)
                data = self.notch_filter(data=data, fs=fs, freq=60.0, Q=30.0)
                data = self.dwt_denoise_reconstruct(signal=data, wavelet='db4', level=3, mode='soft')
                filtered_row[channel] = ','.join(map(lambda v: f"{v:.6f}", data))
            filtered.append(filtered_row)
        df_filtered = pd.DataFrame(filtered)
        df_filtered.to_csv(output_path, index=False, sep=';')
        print("Filtragem + DWT concluídas e CSV salvo em:", output_path)

if not os.path.exists(PREPROCESSED_MUSE_PATH):
    Preprocessing().execute(
        input_path=CSV_MUSE_PATH, 
        output_path=PREPROCESSED_MUSE_PATH
    )
else:
    print(f"Arquivo pré-processado já existe em: {PREPROCESSED_MUSE_PATH}")

Arquivo pré-processado já existe em: ../datalake/processed/Muse-v1.0/MU_filtered.csv


## 4. Carregamento dos Dados para o Modelo

Leitura do arquivo CSV pré-processado e transformação dos dados em um formato adequado para o treinamento do modelo de deep learning. Cada amostra é convertida em um array NumPy com shape `(n_amostras, TARGET_LEN, n_canais)`.

In [4]:
def ler_csv(file_path):
    CHANNELS = ['FP1', 'FP2', 'TP10', 'TP9']
    TARGET_LEN = 440

    df = pd.read_csv(file_path, sep=';')
    df = df[df['code'] != -1]

    X_list = []
    y_list = []

    for _, row in df.iterrows():
        channels_data = []
        for ch in CHANNELS:
            arr = np.array([float(x) for x in row[ch].split(',')])
            if len(arr) > TARGET_LEN:
                arr = arr[:TARGET_LEN]
            elif len(arr) < TARGET_LEN:
                arr = np.pad(arr, (0, TARGET_LEN - len(arr)), mode='constant')
            channels_data.append(arr)

        sample = np.stack(channels_data, axis=1)
        X_list.append(sample)
        y_list.append(int(row['code']))

    X = np.stack(X_list, axis=0)
    y = np.array(y_list, dtype=int)

    return X, y

X, y = ler_csv(PREPROCESSED_MUSE_PATH)

## 5. Normalização e Divisão dos Dados

Normalização dos dados utilizando Z-score seguido por `MinMaxScaler` para escalar os valores entre 0 e 1. Em seguida, os dados são divididos em conjuntos de treino, validação e teste.

In [5]:
def normalize(X_train: np.ndarray, X_val: np.ndarray, X_test: np.ndarray):
    mu = X_train.mean(axis=(0, 1), keepdims=True)
    sigma = X_train.std(axis=(0, 1), keepdims=True)
    sigma[sigma == 0] = 1.0

    X_train_z = (X_train - mu) / sigma
    X_val_z   = (X_val - mu) / sigma
    X_test_z  = (X_test - mu) / sigma

    X_train_final = np.zeros_like(X_train_z)
    X_val_final   = np.zeros_like(X_val_z)
    X_test_final  = np.zeros_like(X_test_z)

    n_channels = X_train.shape[2]

    for ch in range(n_channels):
        scaler = MinMaxScaler(feature_range=(0, 1))
        X_train_final[:, :, ch] = scaler.fit_transform(X_train_z[:, :, ch])
        X_val_final[:, :, ch]   = scaler.transform(X_val_z[:, :, ch])
        X_test_final[:, :, ch]  = scaler.transform(X_test_z[:, :, ch])

    return X_train_final, X_val_final, X_test_final

# Divisão dos dados
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Normalização
X_train, X_val, X_test = normalize(X_train, X_val, X_test)

print(f"Treino: {X_train.shape}, Validação: {X_val.shape}, Teste: {X_test.shape}")

Treino: (23904, 440, 4), Validação: (2988, 440, 4), Teste: (2988, 440, 4)


## 6. Construção do Modelo

Definição da arquitetura do modelo, que consiste em uma rede neural recorrente com camadas LSTM bidirecionais, dropout para regularização e camadas densas para a classificação final.

In [6]:
def create_model():
    N_CHANNELS = 4
    N_SAMPLES = 440

    model = Sequential([
        Input(shape=(N_SAMPLES, N_CHANNELS)),
        Bidirectional(LSTM(N_SAMPLES, return_sequences=True)),
        Dropout(0.1),
        Bidirectional(LSTM(N_SAMPLES // 2, return_sequences=True)),
        Dropout(0.1),
        Bidirectional(LSTM(N_SAMPLES // 4, return_sequences=True)),
        Dropout(0.1),
        Flatten(),
        Dense(128, activation='elu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

model = create_model()
model.summary()

I0000 00:00:1760610206.578083   11934 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4000 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 440, 880)       │     1,566,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 440, 880)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 440, 440)       │     1,937,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 440, 440)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 440, 220)       │       484,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 440, 220)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 96800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    12,390,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,380,858 (62.49 MB)

 Trainable params: 16,380,858 (62.49 MB)

 Non-trainable params: 0 (0.00 B)

## 7. Treinamento do Modelo

Treinamento do modelo com os dados preparados. São utilizados callbacks para `EarlyStopping` (interromper o treino se a performance não melhorar) e `ModelCheckpoint` (salvar o melhor modelo encontrado durante o treino).

In [7]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    'melhor_modelo.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=0
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10, 
    batch_size=64,
    callbacks=[early_stop, checkpoint],
    verbose=1
)

Epoch 1/10


2025-10-16 07:23:36.905972: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91400


 74/374 ━━━━━━━━━━━━━━━━━━━━ 2:17 457ms/step - accuracy: 0.1000 - loss: 6.6146

KeyboardInterrupt: 

## 8. Avaliação do Modelo

Avaliação da performance do modelo treinado no conjunto de teste. São calculadas métricas como acurácia, precisão, recall e F1-score, além da exibição de um relatório de classificação detalhado por classe.

In [ ]:
def validate(model, X_test, y_test):
    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro')
    rec = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    print(f"\n📊 Desempenho no conjunto de teste:")
    print(f"Acurácia: {acc:.4f}")
    print(f"Precisão (macro): {prec:.4f}")
    print(f"Recall (macro): {rec:.4f}")
    print(f"F1-score (macro): {f1:.4f}")

    print("\nRelatório por classe:")
    print(classification_report(y_test, y_pred, digits=4))

validate(model, X_test, y_test)